In [ ]:
%load_ext autoreload
%autoreload 2
import dt4dds_benchmark
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

In [ ]:
def mass_to_copynumber(mass,length):
    mass = mass*1e-09
    moles = (mass/(length*615.96 + 36.04))
    return moles*6.022e23


def calculate_coverage(weight,len):
    copynumber = mass_to_copynumber(weight,len)
    pool_seq = 11293
    return(copynumber/pool_seq)

In [ ]:
calculate_coverage(1, 170)

In [ ]:
cal_df = []
for scenario, standard_conc in (('bestcase', 0.04995), ('worstcase', 0.05044)):
    df = pd.read_csv(f"./qPCR_data/{scenario}/calibration.csv")
    df['scenario'] = scenario
    cal_df.append(df)
    
cal_df = pd.concat(cal_df).reset_index(drop=True)
cal_df = cal_df.drop(cal_df.loc[cal_df['Name']=='NTC'].index)
cal_df['conc'] = 10**cal_df['Name'].astype(float) * standard_conc # exact conc in ng/ul based on dilution volumes
cal_df['coverage'] = cal_df.apply(lambda x: calculate_coverage(5*x['conc'], 170), axis=1) # 5 ul per well

cal_df

In [ ]:
fig = px.scatter(
    cal_df,
    y='Cq',
    x='coverage',
    color='scenario',
    log_x=True,
    trendline='ols',
    trendline_options=dict(log_x=True),
)

fig.update_layout(
    width=300,
    height=300,
    margin=dict(l=10, r=10, t=10, b=0),
    showlegend=False,
)

fig.update_xaxes(title_text="Physical redundancy", dtick=1)
fig.update_yaxes(title_text="Cycle threshold")

fig.add_annotation(
    x=0.1,
    y=0.1,
    xref='paper',
    yref='paper',
    showarrow=False,
    text=f"Bestcase scenario<br>Cq = {px.get_trendline_results(fig).px_fit_results.iloc[0].params[1]:.2f} ⋅ log10(Coverage) + {px.get_trendline_results(fig).px_fit_results.iloc[0].params[0]:.1f}<br>R<sup>2</sup> = {px.get_trendline_results(fig).px_fit_results.iloc[0].rsquared:.4f}",
)

fig.add_annotation(
    x=0.95,
    y=0.9,
    xref='paper',
    yref='paper',
    showarrow=False,
    text=f"Worstcase scenario<br>Cq = {px.get_trendline_results(fig).px_fit_results.iloc[1].params[1]:.2f} ⋅ log10(Coverage) + {px.get_trendline_results(fig).px_fit_results.iloc[1].params[0]:.1f}<br>R<sup>2</sup> = {px.get_trendline_results(fig).px_fit_results.iloc[1].rsquared:.4f}",
)

fig = dt4dds_benchmark.analysis.plotting.standardize_plot(fig)
fig.write_image("./figures/calibration_curve.svg")
fig.write_image("./figures/calibration_curve.png", scale=2)
fig.show()
display(px.get_trendline_results(fig).px_fit_results.iloc[0].summary())
display(px.get_trendline_results(fig).px_fit_results.iloc[1].summary())

In [ ]:
def coverage_2_conc(cover):
    copy = cover*11293
    moles = copy/(6.022e23)
    mass = moles*(170*615.96+36.04)
    mass = mass/(1e-09) #mass in ng
    conc = mass/5 #assuming mass in 5uL
    return conc


for index, covs, stock in ((0, [1000,25,10,5,2], [-1,-2,-3,-3,-4]), (1, [1000,50,25,10,5], [-1,-2,-2,-3,-3])):

    coefficients = px.get_trendline_results(fig).px_fit_results.iloc[index].params
    def coverage_2_CT(desired_coverage):
        slope = coefficients[1]
        inter = coefficients[0]
        return inter + np.log10(desired_coverage)*slope

    def CT_2_coverage(input_ct):
        slope = coefficients[0]
        inter = coefficients[1]
        res = pow(10,((input_ct-inter)/slope))
        return res

    def c1v1_c2v2(c1,c2,v2):
        return c2*v2/(c1)
    corr_conc = []
    corr_ct = []
    volume_stock = []
    volume_h2o = []
    dil_factor = []

    for value in covs:
        corr_conc.append(coverage_2_conc(value))
        corr_ct.append(round(coverage_2_CT(value),2))

    for i in range(0,5):
        volume_stock.append(round(c1v1_c2v2(0.05*pow(10,stock[i]),corr_conc[i],100),1)) #stock needed for 100uL of sample
        dil_factor.append(round((0.05*pow(10,stock[i]))/corr_conc[i],1))
        

    for thing in volume_stock:
        volume_h2o.append(100-thing) #water needed for 100uL of sample


    df2_expected = pd.DataFrame(corr_ct, columns=["Expected Ct"])
    df2_expected["Coverage"] = covs
    df2_expected["Concentration [ng/uL]"] = corr_conc
    df2_expected["Stock Solution"] = stock
    df2_expected["Stock Volume [uL]"] = volume_stock
    df2_expected["H2O Volume [uL]"] = volume_h2o
    df2_expected["Dilution Factor"] = dil_factor
    columns_titles = ["Coverage","Concentration [ng/uL]","Expected Ct","Stock Solution","Stock Volume [uL]","H2O Volume [uL]","Dilution Factor"]
    df2_expected = df2_expected.reindex(columns=columns_titles)
    display(df2_expected)

In [ ]:
def Cq_to_coverage(Cq, intercept, slope):
    return 10**((Cq-intercept)/slope)

cq_df = []
for index, scenario in ((0, 'bestcase'), (1, 'worstcase')):
    df = pd.read_csv(f"./qPCR_data/{scenario}/control.csv")
    df['scenario'] = scenario
    df['Cq_corr'] = df['Cq'] + (cal_df.loc[(cal_df.Name == '-3') & (cal_df.scenario == scenario)]['Cq'].mean() - df.loc[df.Name == '-3']['Cq'].mean())
    df['coverage'] = df.apply(lambda x: Cq_to_coverage(x['Cq_corr'], *px.get_trendline_results(fig).px_fit_results.iloc[index].params), axis=1)
    cq_df.append(df)

cq_df = pd.concat(cq_df).reset_index(drop=True)
display(cq_df)

In [ ]:
cq_df.groupby(['Name', 'scenario']).agg(['mean','std']).reset_index().sort_values(by=['scenario', ('coverage', 'mean')])